In [1]:
如何应对IP被封的问题，有几种套路：
修改请求头，模拟浏览器（而不是代码去直接访问）去访问
采用代理IP并轮换
设置访问时间间隔

如何获取代理IP地址：
从该网站获取： https://www.xicidaili.com/
inspect -> 鼠标定位：
要获取的代理IP地址，属于class = "odd"标签的内容：代码如下，获取的代理IP保存在proxy_ip_list列表中
使用代理
proxies的格式是一个字典：
{‘http’: ‘http://IP:port‘,‘https’:'https://IP:port‘}
把它直接传入requests的get方法中即可
web_data = requests.get(url, headers=headers, proxies=proxies)
 
确认代理IP地址有效性
无论是免费还是收费的代理网站，提供的代理IP都未必有效，我们应该验证一下，有效后，再放入我们的代理IP池中，以下通过几种方式：访问网站，得到的返回码是200真正的访问某些网站，获取title等，验证title与预计的相同访问某些可以提供被访问IP的网站，类似于“查询我的IP”的网站，查看返回的IP地址是什么验证返回码

关于http和https代理
可以看到proxies中有两个键值对：
{‘http’: ‘http://IP:port‘,‘https’:'https://IP:port‘}
其中 HTTP 代理，只代理 HTTP 网站，对于 HTTPS 的网站不起作用，也就是说，用的是本机 IP，反之亦然。
我刚才使用的验证的网站是https://jsonip.com, 是HTTPS网站所以探测到的有效代理中，如果是https代理，则返回的是代理地址
如果是http代理，将使用本机IP进行访问，返回的是我的公网IP地址
 

In [2]:
#代码实现
def send_request(page):
    print('=========正在抓取第{}页=========='.format(page))
    #发生请求，响应数据的方法
    base_url = 'https://www.kuaidaili.com/free/inha/{}/'.format(page)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    response=requests.get(base_url,headers=headers)
    data=response.text
    #with open('D:/python/代理.html','wb') as f:
        #f.write(data)
    time.sleep(1)  #爬取速度太快，需要休息一秒。不然反应不过来
    return data
def parse_data(data):
    # 解析数据
    #1 数据转换
    html_data=parsel.Selector(data)
    print(html_data)
    #数据解析
    parse_list=html_data.xpath('//table[@class="table table-bordered table-striped"]/tbody/tr')
    print(parse_list)
    return parse_list
 def check_ip(proxies_list):
    '检测代理ip的方法'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    can_use=[]
    for proxies in proxies_list:
        try:
            response = requests.get('https://www.baidu.com', headers=headers,proxies=proxies,timeout=0.1)
            if response.status_code==200:
                can_use.append(proxies)
        except:
            continue
    print(can_use)
    return can_use


if __name__ =="__main__":
    proxies_list=[]
    for page in range(1,5):
        data=send_request(page)
        parse_list=parse_data(data)
        for tr in parse_list:
            proxies_dict={}
            http_type=tr.xpath('./td[4]/text()').extract_first()
            ip_num=tr.xpath('./td[1]/text()').extract_first()
            port_num=tr.xpath('./td[4]/text()').extract_first()
            proxies_dict[http_type]=ip_num + ":" + port_num
            proxies_list.append(proxies_dict)
    print('获取到的ip数量:',len(proxies_list),'个')
    #print(proxies_dict)
    can_use=check_ip(proxies_list)
    print('能用的代理ip',can_use)
    print('能用的代理ip数量',len(can_use))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 22)

In [ ]:
Selenium
Selenium是一个web的自动化测试工具，最初是为网站自动化测试而开发的，Selenium可以直接运行在浏览器上，它支持所有主流的浏览器（包括phantomJS这些无界面的浏览器），可以接受指令，让浏览器自动加载页面。获取需要的数据，甚至页面截屏。

In [ ]:
#基本代码
#1.创建浏览器对象
driver=webdriver.Chrome()
#2请求页面
driver.get('https://www.baidu.com/')
#3页面的基本操作（点击，输入）
driver.find_element_by_id('kw').send_keys('元尊')   #输入
driver.find_element_by_id('su').click()         #点击
#4页面截屏
time.sleep(2)
driver.save_screenshot('yuanzu.png')
#5获取渲染之后的数据
print(driver.page_source)                  #获取的是F12中elements的数据而不是源代码数据
#6还可以查看请求页面的cookies值
print(driver.get_cookies())
#7查看当然的url路径
print(driver.current_url)
#8关闭浏览器
driver.close()    #关闭页面
driver.quit()     #关闭浏览器
————————————————
版权声明：本文为CSDN博主「地球上最后一个直男」的原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/weixin_45568353/java/article/details/105706382